In [31]:
%pip install yfinance==0.2.54
%pip install crewai==0.70.1
%pip install crewai-tools==0.12.1
%pip install langchain==0.2.16
%pip install langchain-community==0.2.17
%pip install langchain-openai==0.1.25
%pip install duckduckgo_search==6.3.0


[notice] A new release of pip available: 22.2.2 -> 25.1.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.2.2 -> 25.1.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.2.2 -> 25.1.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.2.2 -> 25.1.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.2.2 -> 25.1.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.2.2 -

In [68]:
import json
import os
from datetime import datetime, timedelta

from crewai import Task, Agent, Crew, Process
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain_community.tools import DuckDuckGoSearchResults
from crewai_tools import CSVSearchTool

import yfinance as yf
from IPython.display import Markdown

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = ""
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [34]:
csvWalletTool = CSVSearchTool(csv='Wallet/wallet.csv')

/opt/homebrew/lib/python3.10/site-packages/crewai_tools/tools/base_tool.py:93: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  for arg, attribute in self.args_schema.schema()["properties"].items():


In [35]:
customerManager = Agent(
    role="Customer Stocks Manager",
    goal="Get the customer question about the stock {ticket} and search the customer wallet CSV file for the stocks.",
    backstory="""You are the manager of the customer investiments wallet. 
    You are the client first contact and you provide the other analystis with the necessary stock ticket and wallet information.
    """,
    verbose=True,
    llm=llm,
    max_iter=5,
    tools=[csvWalletTool],
    allow_delegation=False,
    memory=True
)

/opt/homebrew/lib/python3.10/site-packages/langchain_core/tools/structured.py:54: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  return self.args_schema.schema()["properties"]


In [36]:
getCustomerWallet = Task(
    description="""Use the customer question and find the {ticket} in the CSV file.
    Provide if the stock is in the customer wallet and if it is, provide the mean price he paid
    and the total numers of stocks owned.
    """,
    expected_output="If the customer owns the stock, provide the mean price paid and the total stock numbers.",
    agent=customerManager
)

In [37]:
stockPriceAnalyst = Agent(
    role="Senior Stock Price Analyst",
    goal="Find the {ticket} stock price and analyses its trends. Compare with the price that the customer paid.",
    backstory="""You are a highly experienced in analyzing the price of specific stocks and
    make predictions about its future price.
    """,
    verbose=True,
    llm=llm,
    max_iter=5,
    allow_delegation=False,
    memory=True
)

In [50]:
# CRIANDO YAHOO FINANCE TOOL
def fetch_stock_price(ticket):
    end_date = datetime.today()
    start_date = end_date - timedelta(days=365)
    stock = yf.download(ticket, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
    return stock

yahoo_finance_tool = Tool(
    name="Yahoo Finance Tool",
    description="Fetches stocks prices for {ticket} from the last year about a specific company from the Yahoo Finance API.",
    func= lambda ticket: fetch_stock_price(ticket)
)


In [48]:
result = fetch_stock_price("AAPL")

2025-05-29 19:53:16,707 - 6435860480 - base.py-base:128 - ERROR: Failed to get ticker 'AAPL' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%***********************]  1 of 1 completed
2025-05-29 19:53:16,719 - 8635316928 - multi.py-multi:171 - ERROR: 
1 Failed download:
2025-05-29 19:53:16,719 - 8635316928 - multi.py-multi:184 - ERROR: ['AAPL']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


In [51]:
getStockPrice = Task(
    description="Analyze the stock {ticket} price history and create a price trend analyses of up, down or sideways",
    expected_output="""Specify the current trend stocks price - Up, down or sideways
    eg. stock='AAPL, price UP'.
    """,
    tools=[yahoo_finance_tool],
    agent=stockPriceAnalyst
)

In [53]:
newsAnalyst = Agent(
    role="News Analyst",
    goal="""Create a short summary of the market news related to the stock {ticket} company.
    Provide a market Fear & Greed Index score about the company.
    For each requested stock asset, specify a number between 0 and 100, where 0 is extreme fear and 100 extreme greed.
    """,
    backstory="""You are highly experienced in  analyzing market trends and news for more than 10 years.
    You are also a master level analyst in the human psychology.
    You understand the news, their titles and information, but  you look at those with a health dose of skeptcism.
    You consider the source of the news articles.
    """,
    verbose=True,
    llm=llm,
    max_iter=5,
    allow_delegation=False,
    memory=True
)

In [55]:
searchTool = DuckDuckGoSearchResults(backend='news', num_results=10)

In [56]:
getNews = Task(
    description=f"""Use the search tool to search news about the stock ticket.
    The current date is {datetime.now()}
    Compose the results into a helpful report.
    """,
    expected_output="""A summary of the overall market and one paragraph summary for the requested asset.
    Include the fear/greed score based on the news. Use format:
    <STOCK TICKET>
    <SUMMARY BASED ON NEWS>
    <FEAR/GREED SCORE>
    """,
    agent=newsAnalyst,
    tools=[searchTool]
)

In [ ]:
stockRecommender = Agent(
    role="Chief Stock Analyst",
    goal="""Get the data from the customer currently stocks, the provided input of stock price trends and the stock news
    to provide a recommendation: Buy, Sell or Hold the stock.""",
    backstory="""You are the leader of the stock analyst team. You have a great performance in the past 20 years in stock recommendation.
    With all your team informations, you are able to provide the best recommendation for the customer to achieve
    the maximum value creation.
    """,
    verbose=True,
    llm=llm,
    max_iter=5,
    allow_delegation=True,
    memory=True
)

In [60]:
recommendStock = Task(
    description="""Use the stock price trend, the stock news report and the customers stock mean price of the {ticket}
    to provide a recommendation: Buy, Sell or Hold. If the previews reports are not well provided you can
    delegate back to the specific analyst to work again in their task.
    """,
    expected_output="""A brief paragraph with the summary of the reasons for recommendation and the
    recommendation itself in one of the three possible outputs: Buy, Sell or Hold. Use the format:
    <SUMMARY OF REASONS> 
    <RECOMMENDATION>
    """,
    agent=stockRecommender,
    context=[getCustomerWallet, getStockPrice, getNews]
)

In [61]:
copyWriter = Agent(
    role="Stock Content Writer",
    goal="""Write an insightful compelling and informative 6 paragraph long newsletter
    based on the stock price report, the news report and the recommendation report.
    """,
    backstory="""You are an unbelievable copy writer that understand complex finantial concepts and
    explain for a dummie audience. You create compelling stories and narratives that resonate with 
    the audience.
    """,
    verbose=True,
    llm=llm,
    max_iter=5,
    allow_delegation=False,
    memory=True
)

In [64]:
writeNewsletter = Task(
    description="""Use the stock price trend, the stock news report and the stock recommendation
    to write the newsletter. Focus on the stock price trend, news, the fear/greed score and the summary reason for the recommendation.
    Include the recommendation in the newsletter.
    """,
    expected_output="""An eloquent 6 paragraph newsletter formatted as Markdown in an easy readable manner. It should contain:
    - Introduction - set the overal picture
    - Main part - provides the meat of the analysis including the stock price trend, news, the fear/greed score and the summary reason for the recommendation.
    - 3 bullets of the main summary reason of the recommendation  
    - Recommendation Summary
    - Recommendation itself  
    """,
    agent=copyWriter,
    context=[getStockPrice, getNews, recommendStock]
)

In [66]:
crew = Crew(
    agents=[customerManager, stockPriceAnalyst, newsAnalyst, stockRecommender, copyWriter],
    tasks=[getCustomerWallet, getStockPrice, getNews, recommendStock, writeNewsletter],
    verbose=True,
    process=Process.hierarchical,
    share_crew=False,
    manager_llm=llm
)

/opt/homebrew/lib/python3.10/site-packages/langchain_core/tools/structured.py:54: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  return self.args_schema.schema()["properties"]


In [75]:
results = crew.kickoff(inputs={'ticket': 'Give your thoughts about Apple stocks'})


/opt/homebrew/lib/python3.10/site-packages/langchain_core/tools/structured.py:54: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  return self.args_schema.schema()["properties"]


# Agent: Crew Manager
## Task: Use the customer question and find the Give your thoughts about Apple stocks in the CSV file.
    Provide if the stock is in the customer wallet and if it is, provide the mean price he paid
    and the total numers of stocks owned.
    
# Agent: Crew Manager
## Task: Analyze the CSV file to gather Apple stock information and customer stock ownership details


# Agent: Crew Manager
## Using tool: Delegate work to coworker
## Tool Input: 
"{\"task\": \"Analyze the CSV file to gather Apple stock information and customer stock ownership details\", \"context\": \"Need to identify if the customer owns Apple stocks, and if so, provide mean price paid and total stock numbers\"}"
## Tool Output: 

Error executing tool. coworker mentioned not found, it must be one of the following options:
- crew manager



# Agent: Crew Manager
## Using tool: Ask question to coworker
## Tool Input: 
"{\"question\": \"Can you provide guidance on analyzing the CSV file to gather App

In [76]:
Markdown(results.raw)

# Newsletter on Apple Stocks Analysis

## Introduction
Welcome to the newsletter on Apple stock analysis. This edition aims to provide you with comprehensive insights into the historical trends, latest news, fear/greed score, and a detailed recommendation for investing in Apple stocks.

## Main Analysis
### Historical Trends
The historical analysis of Apple stock prices reveals a consistent upward trend over the past year. This indicates strong market performance and investor confidence in the company's growth prospects.

### Latest News
Recent news on Apple stocks suggests an upcoming release of a new product line, which has generated positive investor interest. Such developments often have a significant impact on stock prices, making it an important factor to consider for investment decisions.

### Fear/Greed Score
The fear/greed score for Apple stocks currently leans towards greed, reflecting positive market sentiment and investor optimism in the company's future performance.

### Summary Reason for Recommendation
Based on the historical trend analysis, positive news on upcoming product releases, and the prevailing market sentiment, the recommendation for Apple stocks is to **Buy**. The combination of strong historical performance, positive news catalysts, and favorable market sentiment signals a promising outlook for Apple stocks.

### 3 Key Points for Recommendation
1. **Strong Growth Trajectory**: Apple stocks have demonstrated robust growth trends, indicating a promising investment opportunity.
2. **Positive News Catalysts**: The upcoming product line release is expected to drive further growth and investor interest in Apple stocks.
3. **Market Sentiment**: With the fear/greed score leaning towards greed, the market sentiment is optimistic, supporting the recommendation to buy Apple stocks.

## Recommendation Summary
In conclusion, the analysis suggests that investing in Apple stocks at this time could be a prudent decision. The combination of historical performance, positive news catalysts, and market sentiment favor the outlook for Apple stocks.

## Recommendation
**Buy**: Based on the analysis presented, it is recommended to consider investing in Apple stocks for potential growth and returns in the market.

Stay tuned for more updates and insights in the next edition of our newsletter. Thank you for reading!